Notebook purpose:

- Explore what kind of user holds more than 10 active accounts in a month

In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/mdb_eval")
import src.data.aggregators as agg
import src.data.make_data as md
import src.data.selectors as sl
import src.helpers.data as hd
import src.helpers.io as io

fs = s3fs.S3FileSystem(profile="3di")

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

## load data

In [2]:
dft = dfs = io.read_parquet("s3://3di-data-mdb/clean/mdb_111.parquet")
hd.inspect(dft)

shape: (593,133, 35), users: 270


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,111,-114.0,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397323,False,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
1,2012-01-03,111,8.5,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397324,True,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01


## dev

In [3]:
df = agg.num_accounts(dft)
df.head(3)

accounts_active  accounts_total
user_id ym                                      
111     2012-01                4              12
        2012-02                4              12
        2012-03                3              12

In [4]:
df.accounts_active.describe()

count    7303.000000
mean        2.723675
std         2.129612
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        18.000000
Name: accounts_active, dtype: float64

In [8]:
# identify many-accounts users (mau)
maus = df.groupby("user_id").accounts_active.max().gt(10).rename("mau").reset_index()
dft = dft.merge(maus)
dft.head(3)

,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym,mau
0,2012-01-03,111,-114.0,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397323,False,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01,False
1,2012-01-03,111,8.5,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397324,True,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01,False
2,2012-01-03,111,-290.0,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235638,2018-08-08 10:24:00,lloyds bank,savings,NaN,2014-07-18,2015-03-19,397457,False,0.0,True,False,1.0,NaN,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01,False


In [16]:
df.accounts_active.between(0, 100).all()

True

In [14]:
abs_amounts = dft.amount.abs()
abs_amounts.groupby([dft.mau, dft.user_id, dft.ym]).sum().groupby(
    "mau"
).describe().round().T

mau,False,True
count,6792.0,511.0
mean,15392.0,33224.0
std,56338.0,52207.0
min,0.0,294.0
25%,3101.0,13478.0
50%,6346.0,22334.0
75%,12558.0,35076.0
max,1588286.0,524650.0
